In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
file = ".\\data"+ "\\phaseone-tradedata-C-febuary.parquet"

df = pq.read_table(file).to_pandas()

df.replace({'Electrical equipment and mac.hioery':'Electrical equipment and machinery',
           'Industrial mac.hin ery': 'Industrial machinery'}, inplace = True)

df["hs2"] = df.hs4.str[0:2]

In [3]:
df = df[df["high_catagory"] != "not in aggreement"]

In [4]:
my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

end_use = "hs?get=E_COMMODITY,ALL_VAL_YR,E_COMMODITY_SDESC"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time=2019" + "&COMM_LVL=HS2"

url = url + "&CTY_CODE=5700"

r = requests.get(url) 
    
print(r)
    
dfc = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

dfc.columns = r.json()[0]

dfc = dfc.groupby("E_COMMODITY").agg({"E_COMMODITY_SDESC":"first"})

<Response [200]>


In [5]:
dfc.head()

,E_COMMODITY_SDESC
E_COMMODITY,
01,LIVE ANIMALS
02,MEAT AND EDIBLE MEAT OFFAL
03,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES"
04,DAIRY PRODS; BIRDS EGGS; HONEY; ED ANIMAL PR N...
05,"PRODUCTS OF ANIMAL ORIGIN, NESOI"


In [6]:
df.head()

,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge,hs2
time,,,,,,,,,,,,
2013-01-01,5700,279107.0,900211,0.0,-,9002,9002,"Lens,es prism, s mirrors and other optical e...",Optical and medical instruments,Manufactures,both,90
2013-01-01,5700,4007.0,110630,0.0,-,1106,1106,Flour and meal of dried leguminous vegetables ...,Cereals,Agriculture,both,11
2013-01-01,5700,404107.0,130212,0.0,-,1302,1302,Vegetable saps and extracts; pectic substances...,Other agricultural commodities,Agriculture,both,13
2013-01-01,5700,438996.0,841191,0.0,-,8411,8411,"Turboj e,ts turbopropellers and other gas ...",Industrial machinery,Manufactures,both,84
2013-01-01,5700,620979.0,840810,0.0,-,8408,8408,Comp ression -ignition in ternal combusti...,Industrial machinery,Manufactures,both,84


In [7]:
dfhs2 = df.groupby(["hs2","time"]).agg({"china_exports":"sum"})

In [8]:
dfhs2 = dfhs2.merge(dfc, left_on = ["hs2"], right_index = True, how = "left")

dfhs2.reset_index(inplace = True)

dfhs2["E_COMMODITY_SDESC"] = "HS CODE " + dfhs2.hs2 + ", " + dfhs2["E_COMMODITY_SDESC"]

In [9]:
dfhs2.set_index(["E_COMMODITY_SDESC","time"],inplace = True)

In [10]:
dfhs2.rename({"hs2":"hs-code"},axis = 1, inplace = True)

In [11]:
dfhs2

hs-code  china_exports
E_COMMODITY_SDESC                        time                             
HS CODE 01, LIVE ANIMALS                 2013-01-01      01      7136452.0
                                         2013-02-01      01      2969368.0
                                         2013-03-01      01      7543461.0
                                         2013-04-01      01      1601596.0
                                         2013-05-01      01      7695188.0
...                                                     ...            ...
HS CODE 90, OPTIC, PHOTO ETC, MEDIC OR S 2020-10-01      90    422898513.0
                                         2020-11-01      90    422157362.0
                                         2020-12-01      90    477988759.0
                                         2021-01-01      90    395318968.0
                                         2021-02-01      90    346107868.0

[4425 rows x 2 columns]

In [12]:
file = ".\\data"+ "\\phaseone-tradedata-C-febuary.parquet"

df = pq.read_table(file).to_pandas()

df.replace({'Electrical equipment and mac.hioery':'Electrical equipment and machinery',
           'Industrial mac.hin ery': 'Industrial machinery'}, inplace = True)

df["hs2"] = df.hs4.str[0:2]

df = df[df["high_catagory"] != "not in aggreement"]

dfhs4 = df.groupby(["description","time"]).agg({"china_exports":"sum", "hs4": "first"})

dfhs4.reset_index(inplace = True)

dfhs4["description"] = "HS CODE " + dfhs4["hs4"] + ", " + dfhs4["description"]

dfhs4.set_index(["description", "time"], inplace = True)

In [13]:
dfhs4.rename({"hs4":"hs-code"},axis = 1, inplace = True)

In [14]:
dfhs4

china_exports  \
description                                        time                        
HS CODE 2911, Acetals and hemiacetals, whether ... 2013-01-01        76799.0   
                                                   2013-02-01        67971.0   
                                                   2013-03-01        30507.0   
                                                   2013-04-01            0.0   
                                                   2013-05-01        83827.0   
...                                                                      ...   
HS CODE 2817, Zinc oxide; zinc peroxide            2020-10-01       157246.0   
                                                   2020-11-01       123218.0   
                                                   2020-12-01        36671.0   
                                                   2021-01-01        19993.0   
                                                   2021-02-01       143273.0   

                                                              hs-code  
description                                        time                
HS CODE 2911, Acetals and hemiacetals, whether ... 2013-01-01    2911  
                                                   2013-02-01    2911  
                                                   2013-03-01    2911  
                                                   2013-04-01    2911  
                                                   2013-05-01    2911  
...                                                               ...  
HS CODE 2817, Zinc oxide; zinc peroxide            2020-10-01    2817  
                                                   2020-11-01    2817  
                                                   2020-12-01    2817  
                                                   2021-01-01    2817  
                                                   2021-02-01    2817  

[46653 rows x 2 columns]

In [15]:
dfall = dfhs2.append(dfhs4)

In [16]:
test = dfall.index.unique(0).to_list()

In [17]:
sorted(test)

['HS CODE 01, LIVE ANIMALS',
 'HS CODE 0101, Hors,es  ass,es  mules and hinnies, live',
 'HS CODE 0102, Bovine anima,ls  live',
 'HS CODE 0103, Swine, live',
 'HS CODE 0104, Sheep and goa,ts  live',
 'HS CODE 0105, Poultry, live; chickens, ducks, geese, turkeys and guineas',
 'HS CODE 0106, Anima,ls  liv, e  nesoi',
 'HS CODE 02, MEAT AND EDIBLE MEAT OFFAL',
 'HS CODE 0201, Meat of bovine animals, fresh or chilled',
 'HS CODE 0202, Meat of bovine animals, frozen',
 'HS CODE 0203, Meat of swin, e  fresh, chilled, or frozen',
 'HS CODE 0204, Meat of sheep or goats, fresh, chilled or frozen',
 'HS CODE 0205, Meat of hors,es  ass,es  mules or hinnies, fresh, chilled or frozen',
 'HS CODE 0206, Edib  l e offa  l of  bovine animals, swine, shee, p  goa,ts  horses, asses, mules or hinnies, fresh, chilled or frozen',
 'HS CODE 0207, Meat and edible offal. of the poultry of heading010,5 fres,h   chilled or frozen',
 'HS CODE 0208, Other meat and edible meat offal. fre,sh chilled or frozen',
 'H

In [29]:
dfall.loc["HS CODE 72, IRON AND STEEL"]

,hs-code,china_exports
time,,
2013-01-01,72,114103112.0
2013-02-01,72,158804662.0
2013-03-01,72,128598274.0
2013-04-01,72,98804647.0
2013-05-01,72,146780917.0
...,...,...
2020-10-01,72,66403896.0
2020-11-01,72,20493249.0
2020-12-01,72,30452723.0


In [22]:
out_file = ".\\data"+ "\\data.parquet"

pq.write_table(pa.Table.from_pandas(dfall), out_file)

In [23]:
file = ".\\data"+ "\\data.parquet"

df = pq.read_table(file).to_pandas()

df.loc["IRON AND STEEL, "]

KeyError: 'IRON AND STEEL, HS CODE 72'

In [20]:
df.china_exports.max()

3557751166.0

In [21]:
df[df.hs2 == "31"]

,,hs2,china_exports
E_COMMODITY_SDESC,time,,
